<a href="https://colab.research.google.com/github/gorilla15a/Autojudge-Problem-Difficulty-Predictor/blob/main/notebook/auto_judge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import joblib

In [ ]:
df = pd.read_json('/content/problems_data.jsonl', lines=True)

In [ ]:
df.shape

(4112, 8)

In [ ]:
df.columns

Index(['title', 'description', 'input_description', 'output_description',
       'sample_io', 'problem_class', 'problem_score', 'url'],
      dtype='object')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 
 1   description         4112 non-null   object 
 2   input_description   4112 non-null   object 
 3   output_description  4112 non-null   object 
 4   sample_io           4112 non-null   object 
 5   problem_class       4112 non-null   object 
 6   problem_score       4112 non-null   float64
 7   url                 4112 non-null   object 
dtypes: float64(1), object(7)
memory usage: 257.1+ KB


In [ ]:
df['problem_score'].value_counts()

,count
problem_score,
6.3,88
5.8,86
5.7,73
6.1,73
6.9,67
...,...
9.2,11
1.1,9
9.5,7


In [ ]:
df['problem_class'].value_counts()

,count
problem_class,
hard,1941
medium,1405
easy,766


In [ ]:
df['sample_io'].value_counts()

,count
sample_io,
[],30
"[{'input': '', 'output': ''}]",24
"[{'input': '', 'output': ''}, {'input': '', 'output': ''}]",17
"[{'input': '10 5 1 2 3 4 5', 'output': '1'}, {'input': '1 10 1 1 1 1 1 1 1 1 1 1', 'output': '9'}]",2
"[{'input': '2 3 25', 'output': '5'}, {'input': '1 5', 'output': '2'}, {'input': '2 5 10', 'output': '-1'}]",2
...,...
"[{'input': '4 0 0 0 1 1 0 1 1', 'output': '3'}, {'input': '5 0 0 10 0 10 0 11 1 12 2', 'output': '14'}]",1
"[{'input': '5 6 1 4 2 3 1 2 2 5 4 2 5 5 100 100 0 2 1 77 100 77 100 100 0 0', 'output': 'Case 1: 2 4 3 Case 2: 0 Case 3: impossible'}]",1
"[{'input': '5 4 0 1 10 1 2 20 2 3 30 3 4 40 4 6 0 1 1 0 2 1 0 3 1 1 2 1 1 3 1 2 3 1', 'output': 'Case 1: 300 Case 2: 6'}]",1


In [ ]:
print("Total samples:", len(df))
print("\nMissing values per column:\n", df.isnull().sum())

print("\nClass distribution:")
print(df['problem_class'].value_counts())

Total samples: 4112

Missing values per column:
 title                 0
description           0
input_description     0
output_description    0
sample_io             0
problem_class         0
problem_score         0
url                   0
dtype: int64

Class distribution:
problem_class
hard      1941
medium    1405
easy       766
Name: count, dtype: int64


In [ ]:
import pandas as pd
import re
def clean_text(text):
    # Handle missing values
    if text is None or (isinstance(text, float) and pd.isna(text)):
        return ""

    # If text is list or dict as in case of sample.io
    if isinstance(text, (list, dict)):
        text = str(text)

    # Convert to string and clean
    text = str(text).lower()
    text = re.sub(r'\s+', ' ', text)

    return text.strip()


In [ ]:
text_cols = [
    'title',
    'description',
    'input_description',
    'output_description',
    'sample_io'
]
for col in text_cols:
    df[col] = df[col].apply(clean_text)

In [ ]:
df['full_text'] = (
    df['title'] + " " +
    df['description'] + " " +
    df['input_description'] + " " +
    df['output_description'] + " " +
    df['sample_io']
)

In [ ]:
df['full_text'].iloc[0][:]


"uuu unununium (uuu) was the name of the chemical element with atom number 111, until it changed to röntgenium (rg) in 2004. these heavy elements are very unstable and have only been synthesized in a few laboratories. you have just been hired by one of these labs to optimize the algorithms used in simulations. for example, when simulating complicated chemical reactions, it is important to keep track of how many particles there are, and this is done by counting connected components in a graph. currently, the lab has some python code (see attachments) that takes an undirected graph and outputs the number of connected components. as you can see, this code is based on everyone’s favourite data structure union-find1. after looking at the code for a while, you notice that it actually has a bug in it! the code still gives correct answers, but the bug could cause it to run inefficiently. your task is to construct a graph with a given number of vertices and edges where the code runs very slowly

In [ ]:
X_text = df['full_text']
y_class = df['problem_class']
y_score = df['problem_score']

In [ ]:
df[['full_text', 'problem_class', 'problem_score']].to_csv(
    "processed_problems_data.csv",
    index=False)

In [ ]:
df = pd.read_csv("processed_problems_data.csv")
df.head()


,full_text,problem_class,problem_score
0,uuu unununium (uuu) was the name of the chemic...,hard,9.7
1,house building a number of eccentrics from cen...,hard,9.7
2,mario or luigi mario and luigi are playing a g...,hard,9.6
3,the wire ghost žofka is bending a copper wire....,hard,9.6
4,barking up the wrong tree your dog spot is let...,hard,9.6


In [ ]:
from sklearn.model_selection import train_test_split
X = df['full_text']
y_class = df['problem_class']
y_score = df['problem_score']

X_train, X_test, y_train, y_test = train_test_split(
    X, y_class,
    test_size=0.2,
    random_state=42,
    stratify=y_class
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 1),
    min_df=3,
    stop_words='english'
)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
import numpy as np
math_symbols = ['+', '-', '*', '/', '%', '<', '>', '=', '^']
keywords = ['graph', 'dp', 'dynamic programming', 'recursion',
            'optimize', 'complexity', 'tree', 'greedy']

def count_math_symbols(text):
    return sum(text.count(sym) for sym in math_symbols)

def keyword_frequency(text):
    return sum(text.count(k) for k in keywords)

def handcrafted_features(text_series):
    return np.column_stack([
        text_series.apply(len),
        text_series.apply(count_math_symbols),
        text_series.apply(keyword_frequency)
    ])

In [ ]:
X_train_handcrafted = handcrafted_features(X_train)
X_test_handcrafted = handcrafted_features(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_handcrafted_scaled = scaler.fit_transform(X_train_handcrafted)
X_test_handcrafted_scaled = scaler.transform(X_test_handcrafted)

In [ ]:
from scipy.sparse import hstack
X_train_combined = hstack([X_train_tfidf, X_train_handcrafted])
X_test_combined = hstack([X_test_tfidf, X_test_handcrafted])

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

logreg = LogisticRegression(
    max_iter=2000,
    class_weight='balanced',
    C=2.0,               # less regularization
    solver='lbfgs'
)

logreg.fit(X_train_combined, y_train)

y_pred_lr = logreg.predict(X_test_combined)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))

Logistic Regression Accuracy: 0.44106925880923453
              precision    recall  f1-score   support

        easy       0.39      0.56      0.46       153
        hard       0.55      0.51      0.53       389
      medium       0.33      0.28      0.30       281

    accuracy                           0.44       823
   macro avg       0.42      0.45      0.43       823
weighted avg       0.44      0.44      0.44       823



/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


SVM

In [ ]:
from sklearn.svm import LinearSVC

svm = LinearSVC(
    C=0.3,
    max_iter=20000,
    tol=1e-3
)
svm.fit(X_train_tfidf, y_train)
y_pred = svm.predict(X_test_tfidf)
from sklearn.metrics import accuracy_score, classification_report
print("SVM Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


SVM Accuracy: 0.47630619684082626
              precision    recall  f1-score   support

        easy       0.46      0.26      0.33       153
        hard       0.52      0.71      0.60       389
      medium       0.37      0.27      0.31       281

    accuracy                           0.48       823
   macro avg       0.45      0.41      0.41       823
weighted avg       0.46      0.48      0.45       823



In [ ]:
from sklearn.calibration import CalibratedClassifierCV
svm_calibrated = CalibratedClassifierCV(
    svm,
    method='sigmoid',
    cv=3
)
svm_calibrated.fit(X_train_tfidf, y_train)
probs = svm_calibrated.predict_proba(X_test_tfidf)
predicted_class = svm_calibrated.classes_[np.argmax(probs, axis=1)]
confidence = probs.max(axis=1)

In [ ]:
def handcrafted_signals(text):
    length_score = len(text) / 1000          # normalized
    symbol_score = sum(text.count(s) for s in ['+', '-', '*', '/', '%', '<', '>'])
    keyword_score = sum(text.count(k) for k in ['graph', 'dp', 'recursion', 'tree'])
    return length_score, symbol_score, keyword_score


In [ ]:
def adjusted_confidence(base_conf, predicted_class, text):
    length_s, symbol_s, keyword_s = handcrafted_signals(text)

    bonus = 0.0

    if predicted_class == "hard":
        bonus += 0.03 * keyword_s
        bonus += 0.02 * symbol_s
        bonus += 0.01 * length_s

    elif predicted_class == "easy":
        bonus -= 0.02 * keyword_s

    # Clamp to [0,1]
    return min(1.0, max(0.0, base_conf + bonus))

In [ ]:
for i in range(5):
    text = X_test.iloc[i]
    base_conf = confidence[i]
    pred = predicted_class[i]

    final_conf = adjusted_confidence(base_conf, pred, text)

    print(
        f"{pred.upper()} | base={base_conf:.2f} → adjusted={final_conf:.2f}"
    )

HARD | base=0.53 → adjusted=1.00
HARD | base=0.48 → adjusted=0.57
HARD | base=0.59 → adjusted=0.68
HARD | base=0.47 → adjusted=0.51
HARD | base=0.39 → adjusted=0.40


Random Forest Implementation

In [ ]:
tfidf_rf = TfidfVectorizer(
    max_features=1000,   # lower dimension for RF
    min_df=5,
    stop_words='english'
)

X_train_tfidf_rf = tfidf_rf.fit_transform(X_train).toarray()
X_test_tfidf_rf = tfidf_rf.transform(X_test).toarray()


In [ ]:
X_train_rf = np.hstack([X_train_tfidf_rf, X_train_handcrafted_scaled])
X_test_rf = np.hstack([X_test_tfidf_rf, X_test_handcrafted_scaled])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=300,
    max_depth=25,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train_rf, y_train)

y_pred_rf = rf.predict(X_test_rf)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))

Random Forest Accuracy: 0.5078979343863913
              precision    recall  f1-score   support

        easy       0.47      0.41      0.44       153
        hard       0.55      0.79      0.65       389
      medium       0.38      0.17      0.23       281

    accuracy                           0.51       823
   macro avg       0.46      0.46      0.44       823
weighted avg       0.47      0.51      0.47       823



In [ ]:
def predict_problem_difficulty(text):
    vec = tfidf.transform([text])
    # Stage 1: Class
    pred_class = svm.predict(vec)[0]
    # Stage 2: Base confidence
    probs = svm_calibrated.predict_proba(vec)
    base_conf = probs.max()
    # Stage 3: Adjusted confidence
    final_conf = adjusted_confidence(base_conf, pred_class, text)

    return pred_class, base_conf, final_conf


# Regression Model

In [ ]:
X_train, X_test, y_train_reg, y_test_reg = train_test_split(
    df['full_text'],
    y_score,
    test_size=0.2,
    random_state=42
)

In [ ]:
tfidf_rg = TfidfVectorizer(
    max_features=3000,
    min_df=3,
    stop_words='english'
)

X_train_tfidf_rg = tfidf_rg.fit_transform(X_train)
X_test_tfidf_rg = tfidf_rg.transform(X_test)


In [ ]:
def handcrafted_features(text_series):
    return np.column_stack([
        text_series.apply(len),
        text_series.apply(lambda t: sum(t.count(s) for s in ['+', '-', '*', '/', '%', '<', '>'])),
        text_series.apply(lambda t: sum(t.count(k) for k in ['graph', 'dp', 'recursion', 'tree', 'greedy']))
    ])

X_train_hc = handcrafted_features(X_train)
X_test_hc = handcrafted_features(X_test)


In [ ]:
scaler = StandardScaler()

X_train_hc = scaler.fit_transform(X_train_hc)
X_test_hc = scaler.transform(X_test_hc)


In [ ]:
X_train_reg = hstack([X_train_tfidf_rg, X_train_hc])
X_test_reg = hstack([X_test_tfidf_rg, X_test_hc])

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
lr = LinearRegression()
lr.fit(X_train_reg, y_train_reg)

y_pred_lr = lr.predict(X_test_reg)

print("Linear Regression MAE:", mean_absolute_error(y_test_reg, y_pred_lr))
print("Linear Regression RMSE:", np.sqrt(mean_squared_error(y_test_reg, y_pred_lr)))


Linear Regression MAE: 5.506211109924468
Linear Regression RMSE: 7.13336172232453


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(
    n_estimators=300,
    max_depth=20,
    min_samples_leaf=3,
    random_state=42,
    n_jobs=-1
)

rf_reg.fit(X_train_reg.toarray(), y_train_reg)

y_pred_rf = rf_reg.predict(X_test_reg.toarray())

print("Random Forest MAE:", mean_absolute_error(y_test_reg, y_pred_rf))
print("Random Forest RMSE:", np.sqrt(mean_squared_error(y_test_reg, y_pred_rf)))


Random Forest MAE: 1.693159383714818
Random Forest RMSE: 2.0432281194249837


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_reg = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=4,
    random_state=42
)

gb_reg.fit(X_train_reg.toarray(), y_train_reg)

y_pred_gb = gb_reg.predict(X_test_reg.toarray())

print("Gradient Boosting MAE:", mean_absolute_error(y_test_reg, y_pred_gb))
print("Gradient Boosting RMSE:", np.sqrt(mean_squared_error(y_test_reg, y_pred_gb)))


Gradient Boosting MAE: 1.6771521326517882
Gradient Boosting RMSE: 2.017163785659328


In [ ]:
joblib.dump(tfidf_rf, "tfidf_classifier.pkl")
joblib.dump(rf, "rf.pkl")
joblib.dump(gb_reg, "gb_regressor.pkl")
joblib.dump(tfidf_rg,"tfidf_regressor.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']